In [1]:
import requests
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm
import os

class ImprovedBookScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.books_data = []

    def scrape_goodreads_list(self, url, num_books=10):
        """
        Scrape book information from a Goodreads list page
        """
        print(f"Starting to scrape {num_books} books...")
        
        try:
            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all book entries on the page
            book_entries = soup.find_all('tr', {'itemtype': 'http://schema.org/Book'})
            
            if not book_entries:
                print("No book entries found. The page structure might have changed.")
                return []
            
            for book in tqdm(book_entries[:num_books]):
                try:
                    # Extract book title and author directly from the list page
                    title_element = book.find('span', {'itemprop': 'name'})
                    author_element = book.find('a', {'class': 'authorName'})
                    
                    # Get the book's URL for detailed information
                    title_link = book.find('a', {'class': 'bookTitle'})
                    if title_link and 'href' in title_link.attrs:
                        book_url = 'https://www.goodreads.com' + title_link['href']
                        
                        # Print current book being scraped (for debugging)
                        print(f"\nScraping book: {title_element.text.strip() if title_element else 'Unknown Title'}")
                        
                        # Get detailed book information
                        book_info = self.scrape_book_page(book_url)
                        
                        if book_info:
                            self.books_data.append(book_info)
                        
                        # Random delay between requests
                        time.sleep(random.uniform(2, 4))
                    
                except Exception as e:
                    print(f"Error scraping book: {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"Error scraping list page: {str(e)}")
        
        return self.books_data

    def scrape_book_page(self, url):
        """
        Scrape detailed information from a book's page
        """
        try:
            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Print the URL being scraped (for debugging)
            print(f"Scraping URL: {url}")

            cover_img = (
                soup.find('img', {'class': 'ResponsiveImage'}) or  # New Goodreads
                soup.find('img', {'id': 'coverImage'}) or         # Old Goodreads
                soup.find('img', {'class': 'BookCover__image'})   # Another possible class
            )
            
            # Extract the highest quality image URL
            cover_url = None
            if cover_img and 'src' in cover_img.attrs:
                cover_url = cover_img['src']
                # Sometimes Goodreads uses data-src for lazy loading
                if not cover_url or cover_url.endswith('nophoto'):
                    cover_url = cover_img.get('data-src', '')
                
                # Get the largest version of the image
                cover_url = cover_url.replace('._SX98_', '._SX500_').replace('._SY160_', '._SY750_')
        
            
            # Find the book container
            book_container = soup.find('div', {'class': 'BookPage__mainContent'})
            
            if not book_container:
                print("Book container not found. Trying alternative selectors...")
            
            # Extract book information with multiple fallback selectors
            title = (
                soup.find('h1', {'class': 'Text Text__title1'}) or 
                soup.find('h1', {'id': 'bookTitle'})
            )
            
            author = (
                soup.find('span', {'class': 'ContributorLink__name'}) or 
                soup.find('span', {'itemprop': 'name'})
            )
            
            rating = (
                soup.find('div', {'class': 'RatingStatistics__rating'}) or
                soup.find('span', {'itemprop': 'ratingValue'})
            )
            
            summary = (
                soup.find('div', {'class': 'BookPageMetadataSection__description'}) or
                soup.find('div', {'id': 'description'})
            )
            
            # Extract genres (with multiple possible selectors)
            genres = []
            genre_elements = (
                soup.find_all('span', {'class': 'BookPageMetadataSection__genreButton'}) or
                soup.find_all('a', {'class': 'actionLinkLite bookPageGenreLink'})
            )

            # Extract publication year
            # publish_year = None
            # publication_details = soup.find('div', {'class': 'Row'}).get_text(strip=True) if soup.find('div', {'class': 'Row'}) else ''
            # if "Published" in publication_details:
            #     try:
            #         publish_year = [int(s) for s in publication_details.split() if s.isdigit()][0]
            #     except IndexError:
            #         pass  # No valid year found
            
            # Build book information dictionary
            book_info = {
                'title': title.text.strip() if title else None,
                'author': author.text.strip() if author else None,
                'rating': rating.text.strip() if rating else None,
                'summary': summary.text.strip() if summary else None,
                'genres': [g.text.strip() for g in genre_elements] if genre_elements else None,
                # 'publish_year': publish_year,
                'cover_url': cover_url,
                'url': url
            }
            
            # Print extracted info (for debugging)
            print(f"Extracted title: {book_info['title']}")
            print(f"Extracted author: {book_info['author']}")
            # print(f"Extracted publish year: {book_info['publish_year']}")
            
            return book_info
            
        except Exception as e:
            print(f"Error scraping book page {url}: {str(e)}")
            return None

    def save_to_csv(self, filename='books_dataset.csv'):
        """
        Save the scraped data to a CSV file
        """
        if not self.books_data:
            print("No data to save!")
            return
        file_exists = os.path.exists(filename)   
        df = pd.DataFrame(self.books_data)
        df.to_csv(filename, mode='a', header=not file_exists, index=False, encoding='utf-8')
        print(f"\nDataset saved to {filename}")
        print(f"Number of books saved: {len(df)}")
        print("\nFirst few entries:")
        print(df.head())

# # Example usage
# if __name__ == "__main__":
#     scraper = ImprovedBookScraper()
    
#     # Example Goodreads list URL - replace with your actual URL
#     list_url = "https://www.goodreads.com/list/show/11.Best_Crime_Mystery_Books"
    
#     # Scrape books
#     books = scraper.scrape_goodreads_list(list_url, num_books=10)
    
#     # Save to CSV
#     scraper.save_to_csv()

In [12]:
if __name__ == "__main__":
    scraper = ImprovedBookScraper()
    
    # Example Goodreads list URL - replace with your actual URL
    list_url = "https://www.goodreads.com/list/show/6.Best_Books_of_the_20th_Century?page=11"
    
    # Scrape books
    books = scraper.scrape_goodreads_list(list_url, num_books=100)
    
    # Save to CSV
    scraper.save_to_csv()

Starting to scrape 100 books...


  0%|          | 0/100 [00:00<?, ?it/s]


Scraping book: Motherless Brooklyn
Scraping URL: https://www.goodreads.com/book/show/328854.Motherless_Brooklyn
Extracted title: Motherless Brooklyn
Extracted author: Jonathan Lethem


  1%|          | 1/100 [00:11<18:46, 11.38s/it]


Scraping book: Follow the River
Scraping URL: https://www.goodreads.com/book/show/138872.Follow_the_River
Extracted title: Follow the River
Extracted author: James Alexander Thom


...


Scraping book: A Fire Upon the Deep (Zones of Thought, #1)
Scraping URL: https://www.goodreads.com/book/show/77711.A_Fire_Upon_the_Deep
Extracted title: A Fire Upon the Deep
Extracted author: Vernor Vinge


 99%|█████████▉| 99/100 [17:07<00:11, 11.95s/it]


Scraping book: The Blind Owl
Scraping URL: https://www.goodreads.com/book/show/45967.The_Blind_Owl
Extracted title: The Blind Owl
Extracted author: Sadegh Hedayat


100%|██████████| 100/100 [17:17<00:00, 10.37s/it]


Dataset saved to books_dataset.csv
Number of books saved: 100

First few entries:
                      title                author rating  \
0       Motherless Brooklyn       Jonathan Lethem   3.86   
1          Follow the River  James Alexander Thom   4.23   
2         Fantastic Mr. Fox            Roald Dahl   4.08   
3  A Girl of the Limberlost  Gene Stratton-Porter   4.17   
4          Love You Forever         Robert Munsch   4.38   

                                             summary  \
0  Lionel Essrog is Brooklyn’s very own self-appo...   
1  Mary Ingles was twenty-three, happily married,...   
2  Fantastic Mr. Fox is on the run! The three mea...   
3  Set amid Indiana's vast Limberlost Swamp, this...   
4  An extraordinarily different story by Robert M...   

                                              genres  \
0  [Fiction, Mystery, Crime, New York, Novels, Au...   
1  [Historical Fiction, Fiction, Historical, Adve...   
2  [Childrens, Fiction, Fantasy, Classics, Middle..

In [20]:
import pandas as pd
from tabulate import tabulate
import textwrap

class BookDataDisplay:
    def __init__(self, csv_file='books_dataset.csv'):
        self.df = pd.read_csv(csv_file)
        
    def print_basic_info(self):
        """Print basic dataset information"""
        print("\n=== Dataset Overview ===")
        print(f"Total number of books: {len(self.df)}")
        print(f"Columns available: {', '.join(self.df.columns)}")
        print("\nFirst few books:")
        print("----------------")
        
        for idx, row in self.df.head().iterrows():
            print(f"\nBook {idx + 1}:")
            print(f"Title: {row['title']}")
            print(f"Author: {row['author']}")
            if 'rating' in row:
                print(f"Rating: {row['rating']}")
            print("-" * 50)
    
    def print_detailed_book(self, index):
        """Print detailed information for a specific book"""
        try:
            book = self.df.iloc[index]
            print("\n=== Detailed Book Information ===")
            print(f"Title: {book['title']}")
            print(f"Author: {book['author']}")
            if 'rating' in book:
                print(f"Rating: {book['rating']}")
            if 'genres' in book:
                print(f"Genres: {book['genres']}")
            print("\nSummary:")
            if 'summary' in book and isinstance(book['summary'], str):
                # Wrap summary text for better readability
                wrapped_summary = textwrap.fill(book['summary'], width=70)
                print(wrapped_summary)
            print("-" * 50)
        except IndexError:
            print(f"Error: Book index {index} not found in dataset")
    
    def print_tabulated(self, columns=None, max_rows=10):
        """Print data in a neat table format"""
        if columns is None:
            columns = ['title', 'author', 'rating']
        
        # Select only existing columns
        available_columns = [col for col in columns if col in self.df.columns]
        
        print("\n=== Books Table ===")
        print(tabulate(
            self.df[available_columns].head(max_rows), 
            headers='keys', 
            tablefmt='grid',
            showindex=True
        ))
    
    def search_books(self, keyword, column='title'):
        """Search for books containing a keyword in specified column"""
        if column not in self.df.columns:
            print(f"Error: Column '{column}' not found in dataset")
            return
        
        matches = self.df[self.df[column].str.contains(keyword, case=False, na=False)]
        
        print(f"\n=== Search Results for '{keyword}' in {column} ===")
        print(f"Found {len(matches)} matches:")
        
        for idx, book in matches.iterrows():
            print(f"\nMatch {idx + 1}:")
            print(f"Title: {book['title']}")
            print(f"Author: {book['author']}")
            if 'rating' in book:
                print(f"Rating: {book['rating']}")
            print("-" * 30)

# Example usage
if __name__ == "__main__":
    display = BookDataDisplay('books_dataset.csv')
    
    # Print basic dataset information
    display.print_basic_info()
    
    # # Print detailed information for first book
    display.print_detailed_book(0)
    
    # # Print tabulated data
    # display.print_tabulated()
    
    # # Search for books
    # display.search_books('fantasy', 'genres')


=== Dataset Overview ===
Total number of books: 5
Columns available: title, author, rating, summary, genres, url

First few books:
----------------

Book 1:
Title: The Girl with the Dragon Tattoo
Author: Stieg Larsson
Rating: 4.17
--------------------------------------------------

Book 2:
Title: And Then There Were None
Author: Agatha Christie
Rating: 4.28
--------------------------------------------------

Book 3:
Title: Angels & Demons
Author: Dan    Brown
Rating: 3.95
--------------------------------------------------

Book 4:
Title: Rebecca
Author: Daphne du Maurier
Rating: 4.25
--------------------------------------------------

Book 5:
Title: In Cold Blood
Author: Truman Capote
Rating: 4.09
--------------------------------------------------

=== Detailed Book Information ===
Title: The Girl with the Dragon Tattoo
Author: Stieg Larsson
Rating: 4.17
Genres: ['Fiction', 'Mystery', 'Thriller', 'Crime', 'Mystery Thriller', 'Suspense', 'Contemporary']

Summary:
Harriet Vanger, a scio